In [1]:
import requests
import json
import pandas as pd
import folium
import warnings
(warnings.filterwarnings('ignore'))
## import all the modules needed for this program


##the function help the user to find find the perfect recipe they want
def recipefinder(food, calories, amount, health_options):
    
    ##this part is when the user have input the allergies and the system refined the outcome
    if health_options.lower() == "no": 
        url = "https://api.edamam.com/search?q="+food+"&app_id=a286e8a0&app_key=4aaa2a04babddd64227681ca7b36502c&from=0&to="+amount+"&calories="+calories
        response = requests.get(url)
        recipes = response.json()
        return recipes
    
    ##this part is when the user have no allergies and showing the unrefined result
    else: 
        url ="https://api.edamam.com/search?q="+food+"&app_id=a286e8a0&app_key=4aaa2a04babddd64227681ca7b36502c&from=0&to="+amount+"&calories="+calories+"&excluded="+health_options
        response = requests.get(url)
        recipes = response.json()
        return recipes 
    
##this function helped to find the nearby grocery store, after they know the ingredients they need for the recipe
##we are using YELP Api to load the map
def nearbygrocerystores(location):
    params = {}
    params["term"] = "grocery store"
    params["location"] =location

    token = "QpgOEI29NHp0IosYkVAneBhWrIKJBDpnfKmu1xEDua5Y1exka_FLmXcxzj9Ir7IcBtKMfqAY-QxYujYcTpRvmbeet0hzrU-KPAKD_BbrHiCdjKVVcWoD6R0D0S3vWnYx"
    headers = {'Authorization': 'Bearer %s' % token}

    url = "https://api.yelp.com/v3/businesses/search"
    request = requests.get(url, params=params, headers=headers)

    data = request.json()
    return data

In [2]:
print("The Perfect Recipe Finder")
print("This program will help you find the perfect recipe, taking into account your desired calorie intake limit and dietary restrictions.")
print('-' * 100)
while True: 
    try:
        #the user input values about their basic requirements over a healthy meal
        user_input_food = input("Please enter a ingredient you want to cook: ") ##what kind of ingredient needed
        user_input_calories = input("Please enter your desired calorie limit ex.(500): ") ##the calories ranges needed
        user_input_allergy = input("Are you allergic to anything? If yes, enter the thing you are allergic of. If no, just enter \"no\": ")
        ##user allergy options
        user_input_amount = input("How many options do you want to see? ")
        results = recipefinder(user_input_food, user_input_calories, user_input_amount, user_input_allergy)
        
    #To handle errors that may occur from trying to connect to url
    except json.decoder.JSONDecodeError:
        print("Invalid inputs. Please follow the instructions and try again.")
        continue
    except requests.exceptions.RequestException as e:
        print("Error: Cannot connect. Details:", e, "Trying again...")
        continue
    except requests.exceptions.HTTPError:
        print("Response was not okay. Trying again...")
        continue
    
    #Create a list of user desired recipe and print out
    list_recipes = []
    count = 1
    print("Results: ")
    ##hits is where the api stores all the recipes
    for hit in results['hits']:
        recipe_name = hit['recipe']['label'] ##indictionary hit the list of lable can show the recipes name
        list_recipes.append(recipe_name)
        print(count, recipe_name)
        count += 1
    if list_recipes == []: ##if the user input recipes has no result
        print("No results found. Try again.")
        continue ##continue the loop
    try:
        ## user decide how many results they want to see from the searching
        recipe_choice = input("Indicate which recipe (number) you would like more details on. If you want to restart your search, enter \"r\": ")
        if recipe_choice == 'r': ##create a handle 'r' to exit
            continue
        elif int(recipe_choice) > len(list_recipes): ##limit the user input choice to the range the system have
            print("Please choose a number within the range of provided recipes. Restarting...")
            continue
        else:
            ## user input of whether they want to see the ingredients of their desired recipe
            user_choice_ingredients = input("Do you want to see the ingredients of this recipe? "
                                           "If so, enter \"Yes\", if not, enter any key to continue. ")
            if user_choice_ingredients.lower() == "yes":
                print("Ingredients:")
                print('-' * 100) ## A cutting line so the reslut will be more clear
                for ingredient in results['hits'][int(recipe_choice)-1]['recipe']['ingredientLines']:
                    print("- %s" % ingredient)       
            else: ##if the user choose "no"
                continue
    except ValueError: ## if the user input something other than integer
        print("Please enter an integer or 'r'. ")
        
    ##create the dataframe of nutritional facts
    ##ask for user input of willingness to see the nutritional facts
    user_choice_nutrition = input("Do you want to see the nutritional facts of this recipe? "
                                  "If so, enter \"Yes\", if not, enter any key to continue. ")
    if user_choice_nutrition.lower() == "yes":
        try:
            dic_list = []
            nutrients_list = ['FAT', 'SUGAR', 'PROCNT', 'CA', 'FE', 'MG', 'ZN', 'VITC', 'VITD', 'TOCPHA']
            
            '''
            #this code can see the whole nutrient table, but we don't want to use it
            for nutrient in results['hits'][int(recipe_choice)-1]['recipe']['totalNutrients']:
                dic_list.append(results['hits'][int(recipe_choice)-1]['recipe']['totalNutrients'][nutrient])
            '''
            
            for nutrient in nutrients_list:
                if nutrient in results['hits'][int(recipe_choice)-1]['recipe']['totalNutrients']:
                    dic_list.append(results['hits'][int(recipe_choice)-1]['recipe']['totalNutrients'][nutrient])
                else:
                    new_dic = {}
                    new_dic['label'] = nutrient
                    new_dic['quantity'] = 0
                    new_dic['unit'] = 'none'
                    dic_list.append(new_dic)

            i=1
            nutrition_dict = {}
            for item in dic_list:
                nutrition_dict[str(i)] = pd.Series(data = item)
                i=i+1
            nutrition_df = pd.DataFrame(nutrition_dict)
            nutrition_df = nutrition_df[['1','2','3','4','5','6','7','8','9','10']]
            display(nutrition_df)
            
        except KeyError: ##if the ingredients the user chosen don't contain the common nutrients factor we sorted
            print("We do not have sufficient nutritional facts to show you. Restarting...")
            continue
    else: ## if the user input "no" 
        pass
    print("To view the recipe, visit %s" % results['hits'][int(recipe_choice)-1]['recipe']['shareAs'])
    ## print the recipe straight out
    
    ## here is when the user choose whether to see the nearby grocery stores 
    user_choice_groceries = input("Would you like to find grocery stores nearby? "
                                  "If so, enter \"Yes\", if not, enter any key to quit. ")
    if user_choice_groceries.lower() != "yes": ## when the user input keys that are not yes
        print("Ending program...")
        break ## quit the program
    else: ## when the user entered "yes"
        user_location = input("Where are you right now? ")

        data = nearbygrocerystores(user_location) ## define where the user is located 

        grocery_stores = data['businesses'] ## pull out the list of businesses in the data
        print("Results: ")
        
        for grocery_store in grocery_stores: ## for loop to see nearby grocery 
            print('-'*100) ## cutting line to make the outcome clear
            name = grocery_store['name'] 
            print(name)
            for line in grocery_store['location']['display_address']:  
                print(line)
            print("%s has received a rating of %.1f stars out of 5 with %d ratings." 
                  % (name, grocery_store['rating'], grocery_store['review_count']))
            yelp_url = grocery_store['url']
            print(yelp_url)
            print('-'*100)
        print("To view these stores on a map, run the following cell.")
        break

The Perfect Recipe Finder
This program will help you find the perfect recipe, taking into account your desired calorie intake limit and dietary restrictions.
----------------------------------------------------------------------------------------------------
Please enter a ingredient you want to cook: fish
Please enter your desired calorie limit ex.(500): 1000
Are you allergic to anything? If yes, enter the thing you are allergic of. If no, just enter "no": fish
How many options do you want to see? 1
Results: 
1 Origami Sauce recipes
Indicate which recipe (number) you would like more details on. If you want to restart your search, enter "r": 1
Do you want to see the ingredients of this recipe? If so, enter "Yes", if not, enter any key to continue. yes
Ingredients:
----------------------------------------------------------------------------------------------------
- 3 tablespoons shaoxing rice wine
- 2 tablespoons fish sauce
- 1 1/2 teaspoons sugar
Do you want to see the nutritional fac

,1,2,3,4,5,6,7,8,9,10
label,Fat,Sugars,Protein,Calcium,Iron,Magnesium,Zinc,Vitamin C,VITD,TOCPHA
quantity,0.0036,7.5978,2.0451,17.778,0.32865,65.682,0.08157,0.18,0,0
unit,g,g,g,mg,mg,mg,mg,mg,none,none


To view the recipe, visit http://www.edamam.com/recipe/origami-sauce-recipes-7e3b2405307c7092e1200657cc1fcd2a/fish/1000-cal
Would you like to find grocery stores nearby? If so, enter "Yes", if not, enter any key to quit. yes
Where are you right now? Syracuse
Results: 
----------------------------------------------------------------------------------------------------
Wegmans
3325 W Genesee St
Syracuse, NY 13219
Wegmans has received a rating of 4.0 stars out of 5 with 29 ratings.
https://www.yelp.com/biz/wegmans-syracuse-3?adjust_creative=nd8E-ENnmQPB7dGmYrkHZw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=nd8E-ENnmQPB7dGmYrkHZw
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
Trader Joe's
3422 Erie Blvd E
Syracuse, NY 13214
Trader Joe's has received a rating of 4.5 stars out of 5 with 37 ratings.
https://www.yel

In [3]:
## print out the map for the selected grocery stores
data = nearbygrocerystores(user_location)
center = (data['businesses'][0]['coordinates']['latitude'], data['businesses'][0]['coordinates']['longitude']) 
#center of map

map = folium.Map(location = center, zoom_start = 11)

grocery_stores = data['businesses'] #list of grocery stores

for grocery_store in grocery_stores:
    lat = grocery_store['coordinates']['latitude']
    long = grocery_store['coordinates']['longitude']
    pos = (lat, long) #find the coordinates of each grocery store
    annotation = folium.Popup(grocery_store['name'], parse_html=True) #popup for marker
    marker = folium.Marker(location = pos, popup = annotation)
    map.add_children(marker) #add marker to the map
map